<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

### Classification Analysis
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## I- Importing packages and Loading the excel file

In [1]:
#Importing packages pandas, matplotlib.pyplot, seaborn, numpy, statsmodels
import pandas as pd
import random as rand
import gender_guesser.detector as gender
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import statsmodels.formula.api as smf 
import sklearn.linear_model
from sklearn.model_selection import train_test_split 
#sklearn.linear_model, train_test_split, , KNeighborsRegressor and StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer   
from sklearn.neighbors import KNeighborsClassifier 
#confusion matrix, DecisionTreeClassifier, auc score, classification trees, random forest, tree plots
from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix 
from sklearn.tree import plot_tree  
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier     
from sklearn.ensemble import GradientBoostingClassifier


# setting random seed
rand.seed(a = 327)
#Importing excel file as a Dataframe 
file = "./GOT_character_predictions.xlsx"

got = pd.read_excel(io = file,
                           sheet_name = 0,
                           header = 0,
                           na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND',
                                        '-1.#QNAN', '-NaN', '-nan',
        '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 'NaN', 'n/a',
        'nan', 'null', ' ?'])

got.head(n=5) #displaying the dataframe


,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,popularity,isAlive
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,...,1.0,0.0,0.0,NaN,0,0,NaN,11,0.605351,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,...,NaN,NaN,NaN,1.0,1,1,97.0,1,0.896321,1
2,3,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,...,NaN,NaN,NaN,NaN,0,1,NaN,0,0.267559,1
3,4,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,...,NaN,NaN,NaN,0.0,1,1,23.0,0,0.183946,0
4,5,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,...,NaN,NaN,NaN,1.0,1,1,29.0,0,0.043478,1


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

### Displaying Dictionary for reference

In [2]:
# pulling up data dictionary
got_description = pd.read_excel('./GOT_data_dictionary.xlsx')


# displaying the data dictionary
got_description

,S.No,Character number (by order of appearance)
0,name,Character name
1,title,Honorary title(s) given to each character
2,male,"1 = male, 0 = female"
3,culture,Indicates the cultural group of a character
4,dateOfBirth,Known dates of birth for each character (measu...
5,mother,Character's biological mother
6,father,Character's biological father
7,heir,Character's biological heir
8,house,Indicates a character's allegiance to a house ...
9,spouse,Character's spouse(s)


### Displaying the dataframe dimensions to know how many rows and features there are.

In [3]:
#Displaying the dimensions of the dataframe
print(f"""
Size of Original Dataset
------------------------
Observations: {got.shape[0]}
Features:     {got.shape[1]}
""")



Size of Original Dataset
------------------------
Observations: 1946
Features:     25



<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## II- Data Cleaning and Preparation

Extracting from feature 'name' the first names to guess the gender of GOT characters. First step is to create a new dataframe with only 'name' column to focus our work and not altering original dataframe.

In [4]:
# STEP 1: splitting personal emails

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in got.iterrows():

    # splitting email domain at ' ' (space)
    split_name = got.loc[index, 'name'].split(sep = ' ')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_name)
    

# converting placeholder_lst into a DataFrame 
name_df = pd.DataFrame(placeholder_lst)


# displaying the results
name_df

,0,1,2,3,4,5
0,Viserys,II,Targaryen,None,None,None
1,Walder,Frey,None,None,None,None
2,Addison,Hill,None,None,None,None
3,Aemma,Arryn,None,None,None,None
4,Sylva,Santagar,None,None,None,None
...,...,...,...,...,...,...
1941,Luwin,None,None,None,None,None
1942,Reek,None,None,None,None,None
1943,Symeon,Star-Eyes,None,None,None,None
1944,Coldhands,None,None,None,None,None


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<strong>Next step is to merge the new column obtained to the original dataframe.<strong>

In [5]:
# STEP 2: concatenating with original DataFrame

# safety measure in case of multiple concatenations
got = pd.read_excel(file)


# concatenating personal_email_domain with friends DataFrame
got = pd.concat([got, name_df[0]],
                   axis = 1)

got.iloc[: , -1]

0         Viserys
1          Walder
2         Addison
3           Aemma
4           Sylva
          ...    
1941        Luwin
1942         Reek
1943       Symeon
1944    Coldhands
1945        Tytos
Name: 0, Length: 1946, dtype: object

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<strong>The third step is to guess the gender of each character based on the the first name.<strong> 
<strong>This columns has been set as a comment after use to minimize processing time.<strong> 

In [6]:
# guessing gender based on name

# placeholder list
#placeholder_lst =[]


# looping to guess gender
#for name in got.iloc[: , -1]:
    #guess = gender.Detector().get_gender(name)
   
    #placeholder_lst.append(guess)
   


#converting list into a series
#got['gender_guess'] = pd.Series(placeholder_lst)


# checking results
#got.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<strong>The results from previous cell has been stored as a list, to be further used and transformed into a column.<strong>

In [7]:
gender_guess= ['unknown', 'unknown', 'andy', 'unknown', 'female', 'unknown', 
             'unknown', 'unknown', 'male', 'male', 'mostly_male', 'mostly_male', 
             'mostly_male', 'mostly_male', 'mostly_male', 'mostly_male', 'unknown', 
             'male', 'unknown', 'unknown', 'male', 'male', 'female', 'unknown', 
             'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 
             'andy', 'andy', 'unknown', 'andy', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'female', 'male', 'male', 'unknown', 'male', 'male', 
             'male', 'male', 'male', 'male', 'male', 'mostly_male', 'male', 
             'mostly_male', 'mostly_male', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'female', 'unknown', 'mostly_male', 
             'unknown', 'unknown', 'male', 'female', 'andy', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'mostly_female', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 
             'unknown', 'andy', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 
             'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 
             'male', 'female', 'female', 'female', 'female', 'female', 'female', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 
             'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 
             'mostly_female', 'female', 'unknown', 'mostly_female', 'unknown', 
             'female', 'unknown', 'female', 'unknown', 'male', 'male', 'unknown', 
             'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 
             'unknown', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 
             'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'female', 'male', 'male', 'male', 'male', 'male', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'male', 'unknown', 'unknown', 'female', 'female', 
             'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 
             'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 
             'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'mostly_male', 'female', 'male', 
             'male', 'male', 'female', 'unknown', 'unknown', 'female', 'unknown',
             'unknown', 'male', 'male', 'male', 'female', 'unknown', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 
             'unknown', 'unknown', 'female', 'unknown', 'andy', 'unknown', 
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'female', 'male', 'unknown', 
             'unknown', 'female', 'male', 'unknown', 'male', 'unknown', 'unknown', 
             'male', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'female', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'male', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'female', 'unknown', 'male', 'male', 'unknown',
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 
             'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 
             'male', 'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'male', 'female', 'unknown', 'unknown', 'female', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'female', 'unknown', 'female', 'female', 'female', 'unknown', 
             'unknown', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 
             'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'mostly_female', 'mostly_female', 'mostly_female', 
             'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'female', 'male', 'male',
             'male', 'male', 'unknown', 'female', 'female', 'female', 'unknown',
             'mostly_male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 
             'female', 'male', 'female', 'unknown', 'unknown', 'female', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male',
             'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 
             'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'male', 'female', 'unknown', 'male', 'unknown', 'unknown', 
             'mostly_female', 'male', 'female', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'male', 'female', 'mostly_female', 'female', 'mostly_female', 'mostly_female', 'mostly_female', 'mostly_female', 'mostly_female', 'mostly_female', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'female', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'mostly_male', 'unknown', 'female', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'female', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'mostly_male', 'male', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female',
             'female', 'female', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 
             'male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'male', 'andy', 'male', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'male', 'male', 'male', 'male', 'male', 'male', 'mostly_male', 'mostly_male', 'mostly_male', 'mostly_male', 'unknown', 'unknown', 'unknown', 'male',
             'unknown', 'male', 'male', 'unknown', 'male', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown',
             'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown',
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 
             'mostly_female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'female', 'unknown', 'female', 'unknown', 'female', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 
             'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 
             'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'mostly_male', 'male', 
             'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 
             'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'male', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown']


In [8]:
#converting list into a series
got['gender_guess'] = pd.Series(gender_guess)
got.columns

Index([                      'S.No',                       'name',
                            'title',                    'culture',
                      'dateOfBirth',                     'mother',
                           'father',                       'heir',
                            'house',                     'spouse',
          'book1_A_Game_Of_Thrones',     'book2_A_Clash_Of_Kings',
          'book3_A_Storm_Of_Swords',    'book4_A_Feast_For_Crows',
       'book5_A_Dance_with_Dragons',              'isAliveMother',
                    'isAliveFather',                'isAliveHeir',
                    'isAliveSpouse',                  'isMarried',
                          'isNoble',                        'age',
                 'numDeadRelations',                 'popularity',
                          'isAlive',                            0,
                     'gender_guess'],
      dtype='object')

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Checking if there are any missing values in the dataframe

In [9]:
got.isnull().any().any()

#Checking how many missing values per feature
got.isnull().sum(axis = 0)


S.No                             0
name                             0
title                         1008
culture                       1269
dateOfBirth                   1513
mother                        1925
father                        1920
heir                          1923
house                          427
spouse                        1670
book1_A_Game_Of_Thrones          0
book2_A_Clash_Of_Kings           0
book3_A_Storm_Of_Swords          0
book4_A_Feast_For_Crows          0
book5_A_Dance_with_Dragons       0
isAliveMother                 1925
isAliveFather                 1920
isAliveHeir                   1923
isAliveSpouse                 1670
isMarried                        0
isNoble                          0
age                           1513
numDeadRelations                 0
popularity                       0
isAlive                          0
0                                0
gender_guess                     0
dtype: int64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Proportion of  missing values in the dataframe

In [10]:
got.isnull().mean().round(decimals = 2)

S.No                          0.00
name                          0.00
title                         0.52
culture                       0.65
dateOfBirth                   0.78
mother                        0.99
father                        0.99
heir                          0.99
house                         0.22
spouse                        0.86
book1_A_Game_Of_Thrones       0.00
book2_A_Clash_Of_Kings        0.00
book3_A_Storm_Of_Swords       0.00
book4_A_Feast_For_Crows       0.00
book5_A_Dance_with_Dragons    0.00
isAliveMother                 0.99
isAliveFather                 0.99
isAliveHeir                   0.99
isAliveSpouse                 0.86
isMarried                     0.00
isNoble                       0.00
age                           0.78
numDeadRelations              0.00
popularity                    0.00
isAlive                       0.00
0                             0.00
gender_guess                  0.00
dtype: float64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Checking the type of the features in the dataframe

In [11]:
#Checking the types of the features
got.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   S.No                        1946 non-null   int64  
 1   name                        1946 non-null   object 
 2   title                       938 non-null    object 
 3   culture                     677 non-null    object 
 4   dateOfBirth                 433 non-null    float64
 5   mother                      21 non-null     object 
 6   father                      26 non-null     object 
 7   heir                        23 non-null     object 
 8   house                       1519 non-null   object 
 9   spouse                      276 non-null    object 
 10  book1_A_Game_Of_Thrones     1946 non-null   int64  
 11  book2_A_Clash_Of_Kings      1946 non-null   int64  
 12  book3_A_Storm_Of_Swords     1946 non-null   int64  
 13  book4_A_Feast_For_Crows     1946 

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Flagging missing values

In [12]:
#Flagging missing values: Creation of new columns with for loop
for col in got:
    if got[col].isnull().sum() > 0:
        got['m_'+col] = got[col].isnull().astype(int)
        

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Creation of a new column to group missing values 

In [13]:
#Sum of the missing values into a single new column
got['mv_sum'] = got.iloc[ : , -13: ].sum(axis = 1)


Checking columns in the dataframe 

In [14]:
#checking if missing values columns have been added in the dataframe
got.columns

Index([                      'S.No',                       'name',
                            'title',                    'culture',
                      'dateOfBirth',                     'mother',
                           'father',                       'heir',
                            'house',                     'spouse',
          'book1_A_Game_Of_Thrones',     'book2_A_Clash_Of_Kings',
          'book3_A_Storm_Of_Swords',    'book4_A_Feast_For_Crows',
       'book5_A_Dance_with_Dragons',              'isAliveMother',
                    'isAliveFather',                'isAliveHeir',
                    'isAliveSpouse',                  'isMarried',
                          'isNoble',                        'age',
                 'numDeadRelations',                 'popularity',
                          'isAlive',                            0,
                     'gender_guess',                    'm_title',
                        'm_culture',              'm_dateOfBir

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Checking the unique values of the feature 'fender_guess' 

In [15]:
got["gender_guess"].value_counts(normalize = False,
                                sort= True,
                                ascending= True)

andy               10
mostly_female      21
mostly_male        24
female            125
male              381
unknown          1385
Name: gender_guess, dtype: int64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
'andy' is not a gender in itself. Most of the time, 'andy' names are atributed to male. In GOT, the name 'andy' was given to men. So we will assign 'andy''s gender as 'male'

In [16]:
#TReplacing some values in gender_guess column
got["gender_guess"].replace({"unknown": "mostly_male", "andy": "male"}, inplace=True)
#Displaying new values and their count
got["gender_guess"].value_counts(normalize = False,
                                sort= True,
                                ascending= True)

mostly_female      21
female            125
male              391
mostly_male      1409
Name: gender_guess, dtype: int64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Treating missing values in dataframe to use full dataset into analysis

In [17]:
#making a loop to fill the missing values
for col in got:
    #making a if statement to know if the column is an integer
    if got[col].isnull().sum() > 0 and got[col].dtypes == 'float64':
        #calculating the median of the column
        median = got[col].median()
        #calculating the mean of the column
        mean = round(got[col].mean())
        #making a if statement to know the skewness of the distribution
        if median < mean:
            #function to fill the missing values with the median
            got[col].fillna(value = median,
                         inplace = True)
        elif  mean < median:
            #function to fill the missing values with the mean
            got[col].fillna(value = mean,
                              inplace = True)
        else:
            continue
            
    #analysing if the column is an object
    elif got[col].isnull().sum() > 0 and got[col].dtypes == 'object':
        #calculating the mode
        mode = got[col].mode()
        #print(mode)
        #function to fill the missing values with the mode
        got[col].fillna(value = mode[0],
                     inplace = True)
        
    else: 
        continue
        #continue


In [18]:
#checking if all categorical features have been treated efficiently
got.describe(include = 'object')

,name,title,culture,mother,father,heir,house,spouse,0,gender_guess
count,1946,1946,1946,1946,1946,1946,1946,1946,1946,1946
unique,1946,262,64,17,20,22,347,254,1442,4
top,Viserys II Targaryen,Ser,Northmen,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,Night's Watch,Walder Frey,Jon,mostly_male
freq,1,1392,1393,1927,1922,1925,532,1677,21,1409


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

"Targaryen" are part of the Noble families in GOT. Their title should then be more honorific. 

In [19]:
#Replacing Servant with Noble 
got["title"].replace({"Ser": "Noble"}, inplace=True)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Imputing IsAlive features with zero. We will assume that the missing values may be due to their not-alive status in the Books

In [20]:
# instantiating an imputation value
fill = 0


# imputing 'isAliveMother'
got['isAliveMother'] = got['isAliveMother'].fillna(value = fill)
# imputing 'isAliveFather'
got['isAliveFather'] = got['isAliveFather'].fillna(value = fill)
# imputing 'isAliveSpouse'
got['isAliveSpouse'] = got['isAliveSpouse'].fillna(value = fill)
# imputing 'isAliveHeir'
got['isAliveHeir'] = got['isAliveHeir'].fillna(value = fill)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Checking if all missing values have been identified and well treated

In [21]:
#Checking if any missing values
got.isnull().any().any()

False

In [22]:
#Displaying cleaned dataframe 
got.head(n=5)

,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,m_father,m_heir,m_house,m_spouse,m_isAliveMother,m_isAliveFather,m_isAliveHeir,m_isAliveSpouse,m_age,mv_sum
0,1,Viserys II Targaryen,Noble,Northmen,268.0,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,Night's Watch,Walder Frey,...,0,0,1,1,0,0,0,1,1,7
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Frey,Perra Royce,...,1,1,0,0,1,1,1,0,0,6
2,3,Addison Hill,Noble,Northmen,268.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Swyft,Walder Frey,...,1,1,0,1,1,1,1,1,1,11
3,4,Aemma Arryn,Queen,Northmen,82.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Arryn,Viserys I Targaryen,...,1,1,0,0,1,1,1,0,0,7
4,5,Sylva Santagar,Greenstone,Dornish,276.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Santagar,Eldon Estermont,...,1,1,0,0,1,1,1,0,0,6


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Creating a new binary feature 'Adult', that will consist of columns mother, father, heir and spouse if they are more than 18 year old.

In [23]:
# creating Adult column
got['Adult'] = 0


# looping to append values in Adult column
for index, val in got.iterrows():
    
    # checking for Miss.
    if  got.loc[index, 'mother'] and got.loc[index, 'age']>= 18:
        got.loc[index, 'Adult'] = 1
    elif  got.loc[index, 'father'] and got.loc[index, 'age']>= 18:
        got.loc[index, 'Adult'] = 1
    elif got.loc[index, 'heir'] and got.loc[index, 'age']>= 18:
        got.loc[index, 'Adult'] = 1
    elif got.loc[index, 'spouse'] and got.loc[index, 'age']>= 18:
        got.loc[index, 'Adult'] = 1
        
    
    else:
        continue
        

In [24]:
#checking new column unique values
got['Adult'].value_counts() 

0    1613
1     333
Name: Adult, dtype: int64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### Creation of a new column Child like the Adult column, in the difference that the characters will be under 18. This column is like an antagonist to Adult and will not be used in analysis for multicolinearity reasons. 

In [25]:
# creating Child column
got['Child'] = 0


# looping to find Miss. and Master
for index, val in got.iterrows():
    
    # checking for Miss.
    if  got.loc[index, 'mother'] and got.loc[index, 'age'] <18:
        got.loc[index, 'Child'] = 1
    elif  got.loc[index, 'father'] and got.loc[index, 'age']< 18:
        got.loc[index, 'Child'] = 1
    elif got.loc[index, 'heir'] and got.loc[index, 'age']< 18:
        got.loc[index, 'Child'] = 1
    elif got.loc[index, 'spouse'] and got.loc[index, 'age']< 18:
        got.loc[index, 'Child'] = 1
        
    
    else:
        continue
        

In [26]:
#checking unique values counts (they should be opposite of Adult column unique values counts)
got['Child'].value_counts() 

1    1613
0     333
Name: Child, dtype: int64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### To convert culture column into a discrete variable, the top 5  frequent cultures will be used as reference.

In [27]:
#finding 5 most frequent cultures in GOT
culture_5 = [x for x in got['culture'].value_counts().sort_values(ascending=False).head(5).index]
culture_5

['Northmen', 'Ironborn', 'Free Folk', 'Valyrian', 'Braavosi']

In [28]:
#creating new column culture_5
got['culture_5']=0

# looping to find the top five values in culture column
for index, val in got.iterrows():
    
    # checking for Northmen
    if 'Northmen' in got.loc[ index , 'culture']:
        got.loc[index, 'culture_5'] = 1
        # checking for Ironborn
    elif 'Ironborn' in got.loc[ index , 'culture']:
        got.loc[index, 'culture_5'] = 1
        
    # checking for Free Folk
    elif 'Free Folk' in got.loc[ index , 'culture']:
        got.loc[index, 'culture_5'] = 1
    # checking for Valyrian
    elif 'Valyrian' in got.loc[ index , 'culture']:
        got.loc[index, 'culture_5'] = 1
    # checking for Braavosi
    elif 'Braavosi' in got.loc[ index , 'culture']:
        got.loc[index, 'culture_5'] = 1
    else:
        continue

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

### To convert house column into a discrete variable, the Noble houses will be used as reference. There is a total of 9 noble houses in GOT.

In [29]:
#Creating Royal_House column
got['Royal_House']=0

# looping to find 9 noble houses
for index, val in got.iterrows():
    
    # checking for Targaryen.
    if 'Targaryen' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
        # checking for Lannister
    elif 'Lannister' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
    # checking for Greyjoy
    elif 'Greyjoy' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
# checking for Tyrell
    elif 'Tyrell' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
# checking for Baratheon
    elif 'Baratheon' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
# checking for Martell
    elif 'Martell' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
# checking for Arryn
    elif 'Arryn' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1
        
# checking for Tully       
    elif 'Tully' in got.loc[ index , 'house']:
        got.loc[index, 'Royal_House'] = 1

    else:
        continue

In [30]:
#checking Royal House column unique value counts
got['Royal_House'].value_counts()

0    1671
1     275
Name: Royal_House, dtype: int64

In [31]:
#create the dummy variables for gender_guess
got = pd.get_dummies(data = got, columns = ['gender_guess'])

got.tail(n=5) #displaying dataset

,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,m_age,mv_sum,Adult,Child,culture_5,Royal_House,gender_guess_female,gender_guess_male,gender_guess_mostly_female,gender_guess_mostly_male
1941,1942,Luwin,Noble,Westeros,268.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Stark,Walder Frey,...,1,11,0,1,0,0,0,0,0,1
1942,1943,Reek,Noble,Northmen,268.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Bolton,Walder Frey,...,1,12,0,1,1,0,0,0,0,1
1943,1944,Symeon Star-Eyes,Noble,Northmen,268.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,Night's Watch,Walder Frey,...,1,13,0,1,1,0,0,1,0,0
1944,1945,Coldhands,Noble,Northmen,268.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,Three-eyed crow,Walder Frey,...,1,12,0,1,1,0,0,0,0,1
1945,1946,Tytos Lannister,Casterly Rock,Northmen,220.0,Alyssa Velaryon,Aegon I Targaryen,Jaehaerys Targaryen,House Lannister,Jeyne Marbrand,...,0,7,1,0,1,1,0,0,0,1


In [32]:
# saving results
#got.to_excel('./got.xlsx',
                 #index = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Creating a copy of dataframe with some dropped features.

*S.No has been dropped because it is just a numerotation for the data and has no real meaning for the data and usefulness in prediction.
*Dropping all other categorical values after transforming them into discrete features.

In [33]:
#Creating a copy of Dataframe and droopping some features
got_df= pd.DataFrame.copy(got)
Got= got_df.drop(['S.No','name', 'title', 0, 'title','culture', 'mother', 'father', 'heir', 'house', 'spouse'], axis = 1)
Got.head(n=5)

,dateOfBirth,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,...,m_age,mv_sum,Adult,Child,culture_5,Royal_House,gender_guess_female,gender_guess_male,gender_guess_mostly_female,gender_guess_mostly_male
0,268.0,0,0,0,0,0,1.0,0.0,0.0,0.0,...,1,7,0,1,1,0,0,0,0,1
1,208.0,1,1,1,1,1,0.0,0.0,0.0,1.0,...,0,6,1,0,0,0,0,0,0,1
2,268.0,0,0,0,1,0,0.0,0.0,0.0,0.0,...,1,11,0,1,1,0,0,1,0,0
3,82.0,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0,7,1,0,1,1,0,0,0,1
4,276.0,0,0,0,1,0,0.0,0.0,0.0,1.0,...,0,6,1,0,0,0,1,0,0,0


In [34]:
#Displaying the dimensions of the dataframe
print(f"""
Size of New Dataset
------------------------
Observations: {Got.shape[0]}
Features:     {Got.shape[1]}
""")



Size of New Dataset
------------------------
Observations: 1946
Features:     38



In [35]:
#Checking the type of the features in new Dataframe
Got.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   dateOfBirth                 1946 non-null   float64
 1   book1_A_Game_Of_Thrones     1946 non-null   int64  
 2   book2_A_Clash_Of_Kings      1946 non-null   int64  
 3   book3_A_Storm_Of_Swords     1946 non-null   int64  
 4   book4_A_Feast_For_Crows     1946 non-null   int64  
 5   book5_A_Dance_with_Dragons  1946 non-null   int64  
 6   isAliveMother               1946 non-null   float64
 7   isAliveFather               1946 non-null   float64
 8   isAliveHeir                 1946 non-null   float64
 9   isAliveSpouse               1946 non-null   float64
 10  isMarried                   1946 non-null   int64  
 11  isNoble                     1946 non-null   int64  
 12  age                         1946 non-null   float64
 13  numDeadRelations            1946 

In [36]:
#Descriptive statistics of features 
Got.describe()

,dateOfBirth,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,...,m_age,mv_sum,Adult,Child,culture_5,Royal_House,gender_guess_female,gender_guess_male,gender_guess_mostly_female,gender_guess_mostly_male
count,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,...,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000
mean,559.343782,0.198356,0.374615,0.480473,0.591984,0.395170,0.007708,0.002569,0.007708,0.110483,...,0.777492,10.588900,0.171120,0.828880,0.843268,0.141316,0.064234,0.200925,0.010791,0.724049
std,9236.930558,0.398864,0.484148,0.499747,0.491593,0.489013,0.087479,0.050637,0.087479,0.313571,...,0.416037,1.588053,0.376711,0.376711,0.363641,0.348436,0.245233,0.400795,0.103346,0.447107
min,-28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,268.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,10.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,268.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,11.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,268.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,12.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,298299.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,13.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
#Checking correlation between y-variable and explanatory variables
Got_corr = Got.corr(method = 'pearson').round(decimals = 2)
Got_corr['isAlive'].sort_values(ascending = False)

isAlive                       1.00
book4_A_Feast_For_Crows       0.27
mv_sum                        0.19
m_dateOfBirth                 0.15
m_age                         0.15
m_mother                      0.14
Child                         0.14
m_father                      0.14
m_isAliveFather               0.14
m_isAliveMother               0.14
m_isAliveHeir                 0.13
m_heir                        0.13
gender_guess_female           0.05
m_isAliveSpouse               0.05
m_spouse                      0.05
age                           0.05
m_title                       0.04
m_culture                     0.04
m_house                       0.04
book5_A_Dance_with_Dragons    0.03
culture_5                     0.02
book3_A_Storm_Of_Swords       0.01
gender_guess_male            -0.00
gender_guess_mostly_female    0.00
isAliveSpouse                -0.01
gender_guess_mostly_male     -0.03
isNoble                      -0.04
isAliveFather                -0.04
isMarried           

In [38]:
#Checking unique values counts for y-variable
Got.loc[ : ,'isAlive'].value_counts(normalize = True).round(decimals = 2)

1    0.75
0    0.25
Name: isAlive, dtype: float64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## III- Analysis using Classification

In [39]:
# declaring explanatory variables, dropping variables to avoid multicolinearity
Got_data = Got.drop(['isAlive', 
                       'book3_A_Storm_Of_Swords',
                       'isAliveSpouse', 
                       'gender_guess_male', 'numDeadRelations','isAliveMother', 'isAliveFather',
       'isAliveHeir', 'isAliveSpouse', 'age', 'gender_guess_mostly_female', 
                    'm_spouse','m_isAliveSpouse', 'book5_A_Dance_with_Dragons', 'dateOfBirth',
                    'm_title','m_house' ]
                      , axis = 1)


# declaring response variable
Got_target = Got.loc[ : , 'isAlive']



In [40]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            Got_data,
            Got_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = Got_target) 


# merging training data for statsmodels
Got_train = pd.concat([x_train, y_train], axis = 1)

In [41]:
#checking proportions of varibales in training and testing sest for consistency
print(f"""

Response Variable Proportions (Training Set)
--------------------------------------------
{y_train.value_counts(normalize = True).round(decimals = 2)}



Response Variable Proportions (Testing Set)
--------------------------------------------
{y_test.value_counts(normalize = True).round(decimals = 2)}
""")




Response Variable Proportions (Training Set)
--------------------------------------------
1    0.75
0    0.25
Name: isAlive, dtype: float64



Response Variable Proportions (Testing Set)
--------------------------------------------
1    0.74
0    0.26
Name: isAlive, dtype: float64



In [42]:
#Checking proportion of x and y varibales in training set
print(f"""

Response Variable Proportions (Training Set)
--------------------------------------------
{x_train.shape}



Response Variable Proportions (Training Set)
--------------------------------------------
{y_train.shape}
""")



Response Variable Proportions (Training Set)
--------------------------------------------
(1751, 22)



Response Variable Proportions (Training Set)
--------------------------------------------
(1751,)



In [43]:
#looping to use Got_data values 
for val in Got_data:
    print(f" {val} + ")

 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book4_A_Feast_For_Crows + 
 isMarried + 
 isNoble + 
 popularity + 
 m_culture + 
 m_dateOfBirth + 
 m_mother + 
 m_father + 
 m_heir + 
 m_isAliveMother + 
 m_isAliveFather + 
 m_isAliveHeir + 
 m_age + 
 mv_sum + 
 Adult + 
 Child + 
 culture_5 + 
 Royal_House + 
 gender_guess_female + 
 gender_guess_mostly_male + 


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Logistic Regression (with statsmodel).

This model will be use to ajust x-varibales. It will not be used or displayed in final dynamic string 

In [44]:
# instantiating a logistic regression model object
logistic = smf.logit(formula = """ isAlive ~ 
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book4_A_Feast_For_Crows + 
 mv_sum +
 isNoble + 
 popularity + 
 Adult + 
 culture_5 + 
 Royal_House + 
 gender_guess_female + 
 gender_guess_mostly_male """,
data    = Got_train)



# fitting the model object
results = logistic.fit()


# checking the results SUMMARY
results.summary2()

Optimization terminated successfully.
         Current function value: 0.492907
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Results: Logit
========================================================================
Model:                 Logit              Pseudo R-squared:   0.130     
Dependent Variable:    isAlive            AIC:                1750.1597 
Date:                  2021-12-05 19:14   BIC:                1815.7750 
No. Observations:      1751               Log-Likelihood:     -863.08   
Df Model:              11                 LL-Null:            -992.53   
Df Residuals:          1739               LLR p-value:        3.7990e-49
Converged:             1.0000             Scale:              1.0000    
No. Iterations:        6.0000                                           
------------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
------------------------------------------------------------------------
Intercept                -1.7328   0.7096 -2.4419 0.0146 -3.1236 -0.3420
book1_A_Game_Of_Thrones  -0.5888   0.1588 -3.7081 0.0002 -0.9000 -0.2776
book2_A_Clash_Of_Kings   -0.2968   0.1357 -2.1868 0.0288 -0.5627 -0.0308
book4_A_Feast_For_Crows   1.6113   0.1318 12.2232 0.0000  1.3529  1.8696
mv_sum                    0.2408   0.0592  4.0652 0.0000  0.1247  0.3569
isNoble                   0.1765   0.1378  1.2815 0.2000 -0.0935  0.4466
popularity               -0.9571   0.4180 -2.2900 0.0220 -1.7763 -0.1379
Adult                    -0.2592   0.1979 -1.3097 0.1903 -0.6471  0.1287
culture_5                -0.4280   0.1682 -2.5446 0.0109 -0.7577 -0.0983
Royal_House              -0.2204   0.1675 -1.3163 0.1881 -0.5486  0.1078
gender_guess_female       0.6829   0.2973  2.2970 0.0216  0.1002  1.2656
gender_guess_mostly_male  0.1599   0.1490  1.0730 0.2833 -0.1322  0.4521
========================================================================

"""

In [45]:
# creating a dictionary to store candidate models

candidate_dict = {

 # full model
 'logit_full'   : ['dateOfBirth', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',
       'book4_A_Feast_For_Crows', 'book5_A_Dance_with_Dragons',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isMarried', 'isNoble',
       'age', 'numDeadRelations', 'popularity', 'm_title', 'm_culture',
       'm_dateOfBirth', 'm_mother', 'm_father', 'm_heir', 'm_house',
       'm_spouse', 'm_isAliveMother', 'm_isAliveFather', 'm_isAliveHeir',
       'm_isAliveSpouse', 'm_age', 'mv_sum', 'Adult', 'Child', 'culture_5',
       'Royal_House', 'gender_guess_female', 'gender_guess_mostly_female',
       'gender_guess_mostly_male'],
 

 # significant variables only 
 'logit_sig'    : ['book1_A_Game_Of_Thrones',
 'book2_A_Clash_Of_Kings',
 'book4_A_Feast_For_Crows',
 'mv_sum',
 'isNoble', 
 'popularity', 
 'Adult', 
 'Child', 
 'culture_5',
 'Royal_House', 
 'gender_guess_female',
 'gender_guess_mostly_male']}
    
    


In [46]:
#Checking x-varibales under logit_sig
candidate_dict['logit_sig']

['book1_A_Game_Of_Thrones',
 'book2_A_Clash_Of_Kings',
 'book4_A_Feast_For_Crows',
 'mv_sum',
 'isNoble',
 'popularity',
 'Adult',
 'Child',
 'culture_5',
 'Royal_House',
 'gender_guess_female',
 'gender_guess_mostly_male']

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Logistic Regression (with Scikit model)

In [47]:
# train/test split with the full model
Got_data   =  Got.loc[ : , candidate_dict['logit_sig']]
Got_target =  Got.loc[ : , 'isAlive']

# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            Got_data,
            Got_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = Got_target) # preserving balance


# merging training data for statsmodels
Got_train = pd.concat([x_train, y_train], axis = 1)

# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            random_state = 219)

# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', logreg_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', logreg_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy
# area under the roc curve (auc)
print(roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving AUC score for future use
logreg_auc = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

Training ACCURACY: 0.7796
Testing  ACCURACY: 0.8256
0.6928


C:\Users\Probook G3\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
# confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# Displaying confusion matrix
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 21
False Positives: 29
False Negatives: 5
True Positives : 140



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## KNN Classification

In [49]:
# INSTANTIATING StandardScaler()
scaler = StandardScaler()


# FITTING the data
scaler.fit(Got_data)


# TRANSFORMING the data
x_scaled     = scaler.transform(Got_data)


# converting to a DataFrame
x_scaled_df  = pd.DataFrame(x_scaled) 


# train-test split with the scaled data
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
            x_scaled_df,
            Got_target,
            random_state = 219,
            test_size    = 0.10,
            stratify     = Got_target)


# INSTANTIATING a KNN classification model
knn_opt = KNeighborsClassifier(n_neighbors = 10)


# FITTING the training data
knn_fit = knn_opt.fit(x_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
knn_pred = knn_fit.predict(x_test_scaled)


# SCORING the results
print('Training ACCURACY:', knn_fit.score(x_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', knn_fit.score(x_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4))


# saving scoring data
knn_train_score = knn_fit.score(x_train_scaled, y_train_scaled).round(4)
knn_test_score  = knn_fit.score(x_test_scaled, y_test_scaled).round(4)


# saving AUC score
knn_auc   = roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4)

Training ACCURACY: 0.8115
Testing  ACCURACY: 0.8
AUC Score        : 0.6886


In [50]:
# confusion matrix
knn_tn, \
knn_fp, \
knn_fn, \
knn_tp = confusion_matrix(y_true = y_test, y_pred = knn_pred).ravel()


# Displaying confusion matrix
print(f"""
True Negatives : {knn_tn}
False Positives: {knn_fp}
False Negatives: {knn_fn}
True Positives : {knn_tp}
""")


True Negatives : 23
False Positives: 27
False Negatives: 12
True Positives : 133



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Classification Trees

In [51]:
# INSTANTIATING a classification tree object
tree = DecisionTreeClassifier(max_depth= 8,
                                     min_samples_leaf = 25,
                                     random_state = 219)


# FITTING the training data
tree_fit = tree.fit(x_train, y_train)


# PREDICTING on new data
tree_pred = tree_fit.predict(x_test)


# SCORING the model
print('Tree Training ACCURACY:', tree_fit.score(x_train,
                                                     y_train).round(4))

print('Tree Testing ACCURACY :', tree_fit.score(x_test,
                                                     y_test).round(4))

print('Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = tree_pred).round(4))


# saving scoring data for future use
tree_train_score = tree_fit.score(x_train, y_train).round(4) # accuracy
tree_test_score  = tree_fit.score(x_test, y_test).round(4)   # accuracy


# saving AUC
tree_auc   = roc_auc_score(y_true  = y_test,
                                      y_score = tree_pred).round(4) # auc

Tree Training ACCURACY: 0.8064
Tree Testing ACCURACY : 0.8308
Tree AUC Score: 0.7421


In [52]:
# confusion matrix
tree_tn, \
tree_fp, \
tree_fn, \
tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_pred).ravel()


# Displaying confusion matrix
print(f"""
True Negatives : {tree_tn}
False Positives: {tree_fp}
False Negatives: {tree_fn}
True Positives : {tree_tp}
""")


True Negatives : 28
False Positives: 22
False Negatives: 11
True Positives : 134



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Random Forest

In [53]:
# INSTANTIATING a random forest model with default values
forest = RandomForestClassifier(n_estimators     = 100,
                                    criterion        = "gini",
                                    max_depth        = 7,
                                    min_samples_leaf = 5,
                                    bootstrap        = True,
                                    warm_start       = False,
                                    random_state     = 219)

# FITTING the training data
forest_fit = forest.fit(x_train, y_train)


# PREDICTING based on the testing set
forest_fit_pred = forest_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', forest_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', forest_fit.score(x_test, y_test).round(4))


# saving AUC score
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = forest_fit_pred).round(4))

# saving scoring data for future use
forest_train_score = forest.score(x_train, y_train).round(4) # accuracy
forest_test_score  = forest.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
forest_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = forest_fit_pred).round(4) # auc


Training ACCURACY: 0.815
Testing  ACCURACY: 0.8667
AUC Score        : 0.7531


In [54]:
# confusion matrix
forest_tn, \
forest_fp, \
forest_fn, \
forest_tp = confusion_matrix(y_true = y_test, y_pred = forest_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {forest_tn}
False Positives: {forest_fp}
False Negatives: {forest_fn}
True Positives : {forest_tp}
""")


True Negatives : 26
False Positives: 24
False Negatives: 2
True Positives : 143



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## GBM

In [55]:
# INSTANTIATING the model object without hyperparameters
gbm_default = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.1,
                                              n_estimators  = 100,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 5,
                                              warm_start    = False,
                                              random_state  = 219)


# FIT step is needed as we are not using .best_estimator
gbm_default_fit = gbm_default.fit(x_train, y_train)


# PREDICTING based on the testing set
gbm_default_pred = gbm_default_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', gbm_default_fit.score(x_train, y_train).round(4))
print('Testing ACCURACY :', gbm_default_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_default_pred).round(4))
# saving scoring data for future use
gbm_default_train_score = gbm_default.score(x_train, y_train).round(4) # accuracy
gbm_default_test_score  = gbm_default.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
gbm_default_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = gbm_default_pred).round(4) # auc

Training ACCURACY: 0.8989
Testing ACCURACY : 0.8513
AUC Score        : 0.7624


In [56]:
# unpacking the confusion matrix
gbm_default_tn, \
gbm_default_fp, \
gbm_default_fn, \
gbm_default_tp = confusion_matrix(y_true = y_test, y_pred = gbm_default_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_default_tn}
False Positives: {gbm_default_fp}
False Negatives: {gbm_default_fn}
True Positives : {gbm_default_tp}
""")


True Negatives : 29
False Positives: 21
False Negatives: 8
True Positives : 137



# Hyperparameter Tuned

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Logistic Regression

In [57]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
lr_tuned = LogisticRegression(C            = 1.5000000000000002,
                              warm_start   = True,
                              solver       = 'lbfgs',
                              max_iter     = 1000,
                              random_state = 219)


# FITTING the model to the full dataset
lr_tuned.fit(Got_data, Got_target) # this is ok because already tuned


# PREDICTING based on the testing set
lr_tuned_pred = lr_tuned.predict(x_test)


# SCORING the results
print('LR Tuned Training ACCURACY:', lr_tuned.score(x_train, y_train).round(4))
print('LR Tuned Testing  ACCURACY:', lr_tuned.score(x_test, y_test).round(4))
print('LR Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_tuned_pred).round(4))


# saving scoring data for future use
lr_tuned_train_score = lr_tuned.score(x_train, y_train).round(4) # accuracy
lr_tuned_test_score  = lr_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
lr_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = lr_tuned_pred).round(4) # auc

LR Tuned Training ACCURACY: 0.7801
LR Tuned Testing  ACCURACY: 0.8359
LR Tuned AUC Score        : 0.7128


In [58]:
# unpacking the confusion matrix
lr_tuned_tn, \
lr_tuned_fp, \
lr_tuned_fn, \
lr_tuned_tp = confusion_matrix(y_true = y_test, y_pred = lr_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {lr_tuned_tn}
False Positives: {lr_tuned_fp}
False Negatives: {lr_tuned_fn}
True Positives : {lr_tuned_tp}
""")


True Negatives : 23
False Positives: 27
False Negatives: 5
True Positives : 140



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Classification Tree

In [59]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
tree_tuned = DecisionTreeClassifier(splitter         = 'random',
                                    min_samples_leaf = 7,
                                    max_depth        = 8,
                                    criterion        = 'gini',
                                    random_state     = 219)


# FITTING to the FULL DATASET (due to cross-validation)
tree_tuned_fit = tree_tuned.fit(Got_data, Got_target)


# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned.predict(x_test)


# SCORING the results
print('Training ACCURACY:', tree_tuned.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_tuned.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_tuned_pred).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned.score(x_train, y_train).round(4) # accuracy
tree_tuned_test_score  = tree_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
tree_tuned_auc         = roc_auc_score(y_true  = y_test,
                                       y_score = tree_tuned_pred).round(4) # auc

Training ACCURACY: 0.8132
Testing  ACCURACY: 0.8667
AUC Score        : 0.7793


In [60]:
# unpacking the confusion matrix
tree_tuned_tn, \
tree_tuned_fp, \
tree_tuned_fn, \
tree_tuned_tp = confusion_matrix(y_true = y_test, y_pred = tree_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tree_tuned_tn}
False Positives: {tree_tuned_fp}
False Negatives: {tree_tuned_fn}
True Positives : {tree_tuned_tp}
""")


True Negatives : 30
False Positives: 20
False Negatives: 6
True Positives : 139



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## Random Forest Tuned 

In [61]:
# building a model based on hyperparameter tuning results

# INSTANTIATING with best_estimator
forest_tuned = RandomForestClassifier(criterion='entropy', min_samples_leaf=2,
                       n_estimators=500, random_state=219, warm_start=True)


# FITTING to the FULL DATASET (due to cross-validation)
forest_tuned_fit = forest_tuned.fit(Got_data, Got_target)


# PREDICTING based on the testing set
forest_tuned_pred = forest_tuned_fit.predict(x_test) 


# SCORING the results
print('Forest Tuned Training ACCURACY:', forest_tuned.score(x_train, y_train).round(4))
print('Forest Tuned Testing  ACCURACY:', forest_tuned.score(x_test, y_test).round(4))
print('Forest Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                                       y_score = forest_tuned_pred).round(4))


# saving scoring data for future use
forest_tuned_train_score = forest_tuned.score(x_train, y_train).round(4) # accuracy
forest_tuned_test_score  = forest_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
forest_tuned_auc = roc_auc_score(y_true  = y_test,
                                 y_score = forest_tuned_pred).round(4) # auc

Forest Tuned Training ACCURACY: 0.8829
Forest Tuned Testing  ACCURACY: 0.9179
Forest Tuned AUC Score        : 0.8531


In [62]:
# unpacking the confusion matrix
forest_tuned_tn, \
forest_tuned_fp, \
forest_tuned_fn, \
forest_tuned_tp = confusion_matrix(y_true = y_test, y_pred = forest_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {forest_tuned_tn}
False Positives: {forest_tuned_fp}
False Negatives: {forest_tuned_fn}
True Positives : {forest_tuned_tp}
""")


True Negatives : 26
False Positives: 24
False Negatives: 2
True Positives : 143



<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

## GBM

In [63]:
# INSTANTIATING with best_estimator
gbm_tuned = GradientBoostingClassifier(learning_rate = 0.1,
                                       max_depth     = 3,
                                       n_estimators  = 200,
                                       warm_start    = True,
                                       random_state  = 219)


# FITTING to the FULL DATASET (due to cross-validation)
gbm_tuned_fit = gbm_tuned.fit(Got_data, Got_target)


# PREDICTING based on the testing set
gbm_tuned_pred = gbm_tuned_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', gbm_tuned_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', gbm_tuned_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_tuned_pred).round(4))
# saving scoring data for future use
gbm_tuned_train_score = gbm_tuned.score(x_train, y_train).round(4) # accuracy
gbm_tuned_test_score  = gbm_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
gbm_tuned_auc = roc_auc_score(y_true  = y_test,
                                 y_score = gbm_tuned_pred).round(4) # auc

Training ACCURACY: 0.8458
Testing  ACCURACY: 0.8974
AUC Score        : 0.8131


In [64]:
# unpacking the confusion matrix
gbm_tuned_tn, \
gbm_tuned_fp, \
gbm_tuned_fn, \
gbm_tuned_tp = confusion_matrix(y_true = y_test, y_pred = forest_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_tuned_tn}
False Positives: {gbm_tuned_fp}
False Negatives: {gbm_tuned_fn}
True Positives : {gbm_tuned_tp}
""")


True Negatives : 26
False Positives: 24
False Negatives: 2
True Positives : 143



<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## Final Output

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>


In [65]:
####Printing summary of models and results
print(f"""
Model               Training Accuracy               Testing Accuracy               AUC Score         Confusion Matrix    
-----               ----------------                 -----------------             ----------        -----------------
                                                                                                      tn  fp  fn  tp
                                                                                                      --------------
Logistic              {logreg_train_score}                           {logreg_test_score}                        {logreg_auc}           {logreg_tn, logreg_fp, logreg_fn, logreg_tp}
lr_tuned              {lr_tuned_train_score}                           {lr_tuned_test_score}                        {lr_tuned_auc}           {lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp}
knn                   {knn_train_score}                           {knn_test_score}                           {knn_auc}           {knn_tn, knn_fp, knn_fn, knn_tp}
tree                  {tree_train_score}                           {tree_test_score}                        {tree_auc}           {tree_tn, tree_fp, tree_fn, tree_tp}
tree_tuned            {tree_tuned_train_score}                           {tree_tuned_test_score}                        {tree_tuned_auc}           {tree_tuned_tn, tree_tuned_fp, tree_tuned_fn, tree_tuned_tp}
forest                {forest_train_score}                            {forest_test_score}                        {forest_auc}           {forest_tn, forest_fp, forest_fn, forest_tp}
***forest_tuned       {forest_tuned_train_score}                           {forest_tuned_test_score}                        {forest_tuned_auc}           {forest_tuned_tn, forest_tuned_fp, forest_tuned_fn, forest_tuned_tp}
gbm_default           {gbm_default_train_score}                           {gbm_default_test_score}                        {gbm_default_auc}           {gbm_default_tn, gbm_default_fp, gbm_default_fn, gbm_default_tp}
gbm_tuned             {gbm_tuned_train_score}                           {gbm_tuned_test_score}                        {gbm_tuned_auc}           {gbm_tuned_tn, gbm_tuned_fp, gbm_tuned_fn, gbm_tuned_tp}
""")


Model               Training Accuracy               Testing Accuracy               AUC Score         Confusion Matrix    
-----               ----------------                 -----------------             ----------        -----------------
                                                                                                      tn  fp  fn  tp
                                                                                                      --------------
Logistic              0.7796                           0.8256                        0.6928           (21, 29, 5, 140)
lr_tuned              0.7801                           0.8359                        0.7128           (23, 27, 5, 140)
knn                   0.8115                           0.8                           0.6886           (23, 27, 12, 133)
tree                  0.8064                           0.8308                        0.7421           (28, 22, 11, 134)
tree_tuned            0.8132                  